# AUDIOMATE Corpus

In [1]:
import os

import IPython

import audiomate

from audiomate.corpus import io
from audiomate.corpus import subset

In [2]:
esc50_path = 'output/esc50'

os.makedirs(esc50_path, exist_ok=True)

## Download a corpus
For some of the corpora/datasets there exists a downloader, that can be used to automatically download the needed files. For the corpora that don't have a downloader the files have to be downloaded manually

In [3]:
esc_downloader = io.ESC50Downloader()
esc_downloader.download(esc50_path)

## Read a corpus
To load a corpus you can either use the `load` method on the `Corpus` class and pass the name of the reader you want to use. If no reader is specified the `DefaultReader` will be used.

In [4]:
esc50_corpus = audiomate.Corpus.load(esc50_path, reader='esc-50')

Or you instantiate a specific reader directly.

In [5]:
esc_reader = io.ESC50Reader()
esc50_corpus = esc_reader.load(esc50_path)

### Labels
After the corpus is loaded the corpus can be used. For example to get all labels occuring in the corpus.

In [6]:
labels = esc50_corpus.all_label_values()
print(', '.join(sorted(labels)))

airplane, breathing, brushing_teeth, can_opening, car_horn, cat, chainsaw, chirping_birds, church_bells, clapping, clock_alarm, clock_tick, coughing, cow, crackling_fire, crickets, crow, crying_baby, dog, door_wood_creaks, door_wood_knock, drinking_sipping, engine, fireworks, footsteps, frog, glass_breaking, hand_saw, helicopter, hen, insects, keyboard_typing, laughing, mouse_click, pig, pouring_water, rain, rooster, sea_waves, sheep, siren, sneezing, snoring, thunderstorm, toilet_flush, train, vacuum_cleaner, washing_machine, water_drops, wind


Or to get the number of times a label is occuring in the corpus.

In [7]:
labels = esc50_corpus.label_count()

for label in ['airplane', 'car_horn', 'sheep']:
    print('{:>20} {}'.format(label, labels[label]))

            airplane 40
            car_horn 40
               sheep 40


### Utterances

In [8]:
esc50_corpus.num_utterances

2000

In [9]:
ten_utterances = list(esc50_corpus.utterances.values())[:10]

for utterance in ten_utterances:
    label = utterance.label_lists['default'][0].value
    print('{:<20} {:.2f}s    {}'.format(utterance.idx, utterance.duration, label))

1-100032-A-0         5.00s    dog
1-100038-A-14        5.00s    chirping_birds
1-100210-A-36        5.00s    vacuum_cleaner
1-100210-B-36        5.00s    vacuum_cleaner
1-101296-A-19        5.00s    thunderstorm
1-101296-B-19        5.00s    thunderstorm
1-101336-A-30        5.00s    door_wood_knock
1-101404-A-34        5.00s    can_opening
1-103298-A-9         5.00s    crow
1-103995-A-30        5.00s    door_wood_knock


Reading the audio data.

In [10]:
utterance = esc50_corpus.utterances['1-100038-A-14']

# With the file-native sampling-rate
sampling_rate = utterance.sampling_rate
samples = utterance.read_samples()

print('Sampling-Rate {}, Num-Samples {}'.format(sampling_rate, samples.size))
IPython.display.display(IPython.display.Audio(samples, rate=sampling_rate))

# With a different sampling-rate
sampling_rate = 8000
samples = utterance.read_samples(sr=sampling_rate)

print('Sampling-Rate {}, Num-Samples {}'.format(sampling_rate, samples.size))
IPython.display.display(IPython.display.Audio(samples, rate=sampling_rate))

Sampling-Rate 44100, Num-Samples 220500


Sampling-Rate 8000, Num-Samples 40000


## Filter a corpus
To get a specific subset of a corpus, it can be filtered. For example to filter for specific labels or utterance-ids. This gives a subview that is a view on the original corpus but with the given filters.

In [11]:
filter_criteria = subset.MatchingLabelFilter(labels={'airplane', 'siren', 'train'})
subview = subset.Subview(esc50_corpus, filter_criteria=[filter_criteria])

In [12]:
subview.num_utterances

120

## Save a corpus
For some formats a writer is implemented. In the same way as for reading, the writer has to be specified. If not the `DefaultWriter` is used.

In [14]:
# First we create a normal corpus from the subview
corpus_to_store = audiomate.Corpus.from_corpus(subview)

# Then we store it
os.makedirs('output/esc_subview', exist_ok=True)
corpus_to_store.save_at('output/esc_subview', writer='default')